# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [32]:
import re
import random
from typing import List
from collections import Counter, defaultdict

import nltk
import numpy as np
from tqdm.auto import tqdm


random.seed(0)
np.random.seed(0)

## Nordwig baseline

In [2]:
# code adapted from https://norvig.com/spell-correct.html


def candidates(word, vocab):
    """
    :param word: token to be corrected
    :param vocab: set of known words
    :return: possible spelling corrections for word 
    """
    return known([word], vocab) or known(edits1(word), vocab) or known(edits2(word), vocab) or [word]


def known(words, vocab):
    """
    :param words: list of tokens
    :param vocab: set of known tokens
    :return: subset of words which are in vocab
    """
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in vocab)


def edits1(word):
    """
    :param word: token to be altered
    :return: set of all words within Damerau-Levenstein distance radius 1
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    """
    :param word: token to be altered
    :return: set of all words within Damerau-Levenstein distance radius 2
    """
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


class NordwigCorrector:
    def __init__(self, lines):
        """
        :param lines: list of space-separated tokens
        """
        words = ' '.join(lines).split()
        self.words = Counter(words)
        self.N = sum(self.words.values())

    def P(self, word):
        """
        Estimate the marginal token probability in the known corpora
        :param word: str, a single token 
        :return: P(word)
        """
        return self.words[word] / self.N

    def word_correction(self, word):
        """
        :param word: token with a spelling error
        :return: The correction (within distance of at most 2 from word) with the highest marginal probability
        """
        return max(candidates(word, self.words), key=self.P)

    def correction(self, sentence):
        """
        :param sentence: str, space-separated tokens
        :return: sentence with corrected spelling errors
        """
        words = sentence.lower().split()
        for i in range(len(words)):
            if words[i] not in self.words:
                words[i] = self.word_correction(words[i])
        return ' '.join(words)

## My solution

### N-gram LM

In [5]:
# code has been adapted from https://github.com/yandexdataschool/nlp_course/tree/2023/week03_lm

UNK = "_UNK_"


class KneserNeyLanguageModel:
    def __init__(self, data_or_path, n=3, kn_delta: float = 1.0):
        """
        :param data_or_path: if str given, should be a path to a file with lines having the following structure:
            "occurrence_number word1 word2 ... "
            Otherwise, should be a tuple of two lists: lines of space-separated tokens and occurrence counts for them
        :param n: n for n-gram modeling
        :param kn_delta: delta used in Kneser-Ney smoothing
        """
        assert type(n) is int and 2 <= n <= 5, 'N for N-Grams should be an integer from [2; 5]'
        self.n = n - 1
        self.counts = {}
        if type(data_or_path) is str:
            lines = []
            line_counts = []
            with open(data_or_path, 'r') as dataset:
                for line in dataset.readlines():
                    count, *words = line.strip().split()
                    line_counts.append(int(count))
                    lines.append(' '.join(words))
        else:
            lines, line_counts = data_or_path
        for prefix_len in tqdm(range(self.n + 1), desc='Training LM', leave=False):
            bare_counts = self.count_ngrams(lines, line_counts, prefix_len + 1)
            self.counts[prefix_len] = bare_counts
        self.kn_delta = kn_delta
        self.vocab = set(token for token_counts in self.counts[0].values() for token in token_counts)

    def get_possible_next_tokens(self, prefix, candidates):
        """
        TODO
        :param prefix: str, space-separated tokens
        :param candidates: list of strings - single tokens
        :return: dict { candidate: P(candidate | prefix) }
        """
        return {token: self.get_next_token_prob(prefix, token) for token in candidates}

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: str, space-separated tokens (can be empty)
        :param next_token: str, single token
        :return: P(next_token | prefix)
        """
        prefix = prefix.split()
        p = self.kn_delta / len(self.vocab)
        for prefix_len in range(min(self.n, len(prefix)) + 1):
            key_prefix = tuple(prefix[-prefix_len:]) if prefix_len > 0 else ()
            new = self.counts[prefix_len][key_prefix]
            norm = sum(new.values())
            if norm == 0:
                break
            n_different = len(set([token for token in new.keys() if new[token] > 0]))
            p = max(0, new[next_token] - self.kn_delta) / norm + (p * self.kn_delta * n_different) / norm
        return p

    def correction(self, sentence):
        """
        Iterate over tokens in the sentence, if a word is not in the LM vocab, then iterate over words within 
        Damerau-Levenstein distance of at most 2, rank these words by probability of occurring in the left context
        Choose greedily the word with the largest probability
        :param sentence: str, space-separated tokens
        :return: sentence with corrected spelling errors
        """
        prefix = ''
        words = sentence.split()
        for i in range(len(words)):
            word = words[i]
            if word not in self.vocab:
                dl_closest_words = self.get_possible_next_tokens(prefix, candidates(word, self.vocab)).items()
                largest_likelihood_word = sorted(list(dl_closest_words), key=lambda x: x[1])[-1][0]
                correct_word = largest_likelihood_word
            else:
                correct_word = word
            prefix += correct_word + ' '
        return prefix[:-1]

    @staticmethod
    def count_ngrams(lines, line_counts, n):
        """
        :param lines: an iterable of strings with space-separated tokens
        :param line_counts: list of integers corresponding to line counts
        :param n: n in n-gram to be used when counting
        :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}
        """
        counts = defaultdict(Counter)
        # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
        for t in range(len(lines)):
            line = lines[t].split()
            for i in range(len(line)):
                prefix = []
                if i < n - 1:
                    prefix = [UNK for _ in range(n - 1 - i)]
                prefix.extend(line[max(0, i - n + 1): i])
                counts[tuple(prefix)][line[i]] += line_counts[i]
        return counts

    def perplexity(self, lines, min_log_prob=np.log(10 ** -50.)):
        """
        :param lines: a list of strings with space-separated tokens
        :param min_log_prob: if log(P(w | ...)) is smaller than min_log_prob, set it equal to min_log_prob
        :returns: corpora-level perplexity - a single scalar number
        """
        log_PP = 0
        N = 0
        for line in tqdm(lines, desc='Evaluating perplexity', leave=False):
            prefix = ''
            line_split = line.split()
            N += len(line_split)
            for token in line_split:
                prob = self.get_next_token_prob(prefix, token)
                log_PP -= max(np.log(prob) if prob != 0 else -float('inf'), min_log_prob)
                prefix = prefix + ' ' + token

        return np.exp(log_PP / N)

### Sample usage

In [6]:
kn_lm = KneserNeyLanguageModel('data/fivegrams.txt')

Training LM:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# sanity check
kn_lm.get_possible_next_tokens('there they', ['is', 'are', 'were', 'umbrella'])

{'is': 2.8659434390573218e-06,
 'are': 0.9404379380602723,
 'were': 0.008358404634502927,
 'umbrella': 2.715145139936109e-10}

In [15]:
for sent in ['this tasg is esy fr me', 'due to the lack of foo', 'kicked it with his foo']:
    print(kn_lm.correction(sent))

this task is easy for me
due to the lack of food
kicked it with his foot


### Holmes corpora preparation

In [9]:
holmes_dataset = []

with open('data/big.txt', 'r') as inp:
    text = ''
    for line in inp.readlines():
        if line == '\n':
            if len(text):
                holmes_dataset.append(text[:-1])
            text = ''
        else:
            words = ' '.join(re.findall(r'\w+', line.strip().lower()))
            text += words + ' '

In [10]:
holmes_dataset = [line for line in holmes_dataset if len(line) >= 5]
len(holmes_dataset)

21648

In [11]:
# Generate train-test split
n_train_samples = int(len(holmes_dataset) * 0.9)
train_dataset, test_dataset = holmes_dataset[:n_train_samples], holmes_dataset[n_train_samples:]

In [12]:
# convert the train dataset to format acceptible by KneserNeyLanguageModel
train_dataset_lines_counts = Counter()
for line in train_dataset:
    tokens = line.split()
    for i in range(len(tokens) - 5 + 1):
        train_dataset_lines_counts[' '.join(tokens[i:i+5])] += 1
train_dataset = (list(train_dataset_lines_counts.keys()), list(train_dataset_lines_counts.values()),)

In [13]:
train_dataset[0][:10]

['the project gutenberg ebook of',
 'project gutenberg ebook of the',
 'gutenberg ebook of the adventures',
 'ebook of the adventures of',
 'of the adventures of sherlock',
 'the adventures of sherlock holmes',
 'adventures of sherlock holmes by',
 'of sherlock holmes by sir',
 'sherlock holmes by sir arthur',
 'holmes by sir arthur conan']

### Hyperparameters search

In [14]:
history = []
validation_dataset = np.random.choice(train_dataset[0], 400, replace=False)
for context_size in tqdm(range(2, 4), desc='context iter'):
    for delta in tqdm(range(1, 10 + 1), desc='delta iter', leave=False):
        _lm = KneserNeyLanguageModel(train_dataset, n=context_size, kn_delta = delta / 10)
        pp = _lm.perplexity(validation_dataset)
        history.append((context_size, delta / 10, pp))
for context_size, delta, pp in history:
    print(f'For N={context_size} Delta={delta}: Perplexity = {pp}')

context iter:   0%|          | 0/2 [00:00<?, ?it/s]

delta iter:   0%|          | 0/10 [00:00<?, ?it/s]

Training LM:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating perplexity:   0%|          | 0/400 [00:00<?, ?it/s]

Training LM:   0%|          | 0/2 [00:00<?, ?it/s]

...

Training LM:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating perplexity:   0%|          | 0/400 [00:00<?, ?it/s]

Training LM:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating perplexity:   0%|          | 0/400 [00:00<?, ?it/s]

For N=2 Delta=0.1: Perplexity = 142.31279637520808
For N=2 Delta=0.2: Perplexity = 143.21152642361074
For N=2 Delta=0.3: Perplexity = 144.14151635607982
For N=2 Delta=0.4: Perplexity = 145.10562358365482
For N=2 Delta=0.5: Perplexity = 146.10768472599693
For N=2 Delta=0.6: Perplexity = 147.15323909374723
For N=2 Delta=0.7: Perplexity = 148.2512762906983
For N=2 Delta=0.8: Perplexity = 149.41951560045982
For N=2 Delta=0.9: Perplexity = 150.70846189948583
For N=2 Delta=1.0: Perplexity = 153.3654424198026
For N=3 Delta=0.1: Perplexity = 28.85392635792121
For N=3 Delta=0.2: Perplexity = 29.28912042521709
For N=3 Delta=0.3: Perplexity = 29.74875165498188
For N=3 Delta=0.4: Perplexity = 30.235434063496594
For N=3 Delta=0.5: Perplexity = 30.75246102013715
For N=3 Delta=0.6: Perplexity = 31.30421318910851
For N=3 Delta=0.7: Perplexity = 31.897096249170502
For N=3 Delta=0.8: Perplexity = 32.54227303496812
For N=3 Delta=0.9: Perplexity = 33.26777620325647
For N=3 Delta=1.0: Perplexity = 34.75152

## Justify your decisions

<!-- Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc. -->

- Inside my N-gram LM I used Kneser-Ney smoothing for higher robustness to rare n-grams and overall better generalization than that of no smoothing LM or Laplacian smoothing.
- Optimal value for $\delta$ and $N$ for the LM are found on train dataset by grid search: I have selected the parameters which gave the lowest perplexity on a subset of a training set
- For N-gram dataset I have chosen the fivegrams dataset as one providing extensive statistics over long N-grams which are needed for backoff in Kneser-Ney approach, and for the benchmarking dataset - the Holmes corpora since it was compact and unlike the N-grams dataset a part of the Holmes corpora could be easily separated to be testing set.
- My approach is an improvement of Nordwig's solution: instead of ranking corrections based only on $P(correction)$ derived from the frequency in the dataset, I evaluate $P(correction|context)$ based on the LM trained on the corpus. The statement of improvement is also supported by increased spelling correction top-1 accuracy (see below). The only flaw I have found thus far in my approach is slower inference and training time (almost 2 times slower when inferencing) if compared to Nordwig's solution.

To sum up, Kneser-Ney outperforms the Nordwig's solution significantly in spelling errors correction, however falling short in inference speed. The benefit of LM-based solution is however the ability to make context-sensitive error correction, which can compensate for slower runtime

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [33]:
baseline = NordwigCorrector(train_dataset[0])
best_kn_lm = KneserNeyLanguageModel(train_dataset, n=3, kn_delta=0.1)

Training LM:   0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
def prepare_dataset(lines, vocab):
    corrupted_lines = []
    for line in tqdm(lines):
        words = line.split()
        n_typos = np.random.randint(low=1, high=3 + 1)
        if n_typos > len(words):
            n_typos = 0
        typos_indices = np.random.choice(range(len(words)), size=n_typos, replace=False)
        for typo_idx in typos_indices:
            words[typo_idx] = random_corruption(words[typo_idx], vocab)
        corrupted_lines.append(' '.join(words))
    return corrupted_lines


def random_corruption(word, vocab):
    if np.random.uniform() < 0.2:
        edit_function = edits2
    else:
        edit_function = edits1
    return random.choice([c_word for c_word in edit_function(word) if c_word not in vocab])

In [35]:
test_dataset_corrupt = prepare_dataset(test_dataset, best_kn_lm.vocab)

  0%|          | 0/2165 [00:00<?, ?it/s]

In [36]:
list(zip(test_dataset, test_dataset_corrupt))[:4]

[('i am sure that rascal was lying said the count',
  'i am sfre that rascal was lying said the count'),
 ('they can still be called back said one of his suite who like count orlov felt distrustful of the adventure when he looked at the enemy s camp',
  'they can still be called back said one of his suite who like count orlov felt diutrustful of the adventure when he loored az the enemy s camp'),
 ('eh really what do you think should we let them go on or not',
  'eh reallky what do you think should we lmt them go oin or not'),
 ('will you have them fetched back', 'willvm you have tqhem fetched back')]

In [39]:
def measure_quality(corrector, corrupt, gt, n_samples, results_prefix):
    """
    Measure the ratio of corrected spelling errors to the total number of spelling errors
    :param corrector: either Nordwig model, of KneserNey LM
    :param corrupt: list of space-separated tokens, some of which contain typos
    :param gt: list of correct space-separated tokens sentences
    :param n_samples: Maximal number of lines to consider in evaluation
    :param results_prefix: str, text to be printed before reporting the accuracy
    :return: None, print the accuracy preceded by results_prefix
    """
    corrected_mistakes = 0
    total_mistakes = 0
    for x, y in tqdm(zip(corrupt[:n_samples], gt[:n_samples]), total=n_samples):
        y_hat = corrector.correction(x).split()
        x, y = x.split(), y.split()
        for i in range(len(x)):
            corrected_mistakes += y[i] != x[i] and y[i] == y_hat[i]
            total_mistakes += y[i] != x[i]
        # corrected_mistakes += y == y_hat
        # total_mistakes += 1
    print(f'{results_prefix}: {corrected_mistakes / total_mistakes * 100:.3f}')

In [43]:
measure_quality(baseline, test_dataset_corrupt, test_dataset, n_samples=len(test_dataset), results_prefix='Nordwig solution accuracy')

  0%|          | 0/2165 [00:00<?, ?it/s]

Nordwig solution accuracy: 80.884


In [45]:
measure_quality(best_kn_lm, test_dataset_corrupt, test_dataset, n_samples=len(test_dataset), results_prefix='Kneser-Ney LM accuracy')

  0%|          | 0/2165 [00:00<?, ?it/s]

Kneser-Ney LM accuracy: 85.066
